In [7]:
import torch

# Ybus matrix (3x3)
Y = torch.tensor([
    [20-50j, -10+20j, -10+30j],
    [-10+20j, 26-52j, -16+32j],
    [-10+30j, -16+32j, 26-62j]
], dtype=torch.complex64)

# Per-unit complex powers
S = torch.tensor([0, -256.6 - 110.2j, -138.6 - 45.2j], dtype=torch.complex64) / 100

# Initial guesses
V_mag = torch.tensor([1.05, 1.0, 1.0])
delta = torch.tensor([0.0, 0.0, 0.0])

# Iteration
for it in range(20):
    V = V_mag * torch.exp(1j * delta)
    S_calc = V * (Y @ V).conj()
    mis = torch.cat([(S[1:].real - S_calc[1:].real), (S[1:].imag - S_calc[1:].imag)])

    print(f"\nIter {it+1}:")
    for i in range(3):
        m, a = V[i].abs().item(), torch.rad2deg(torch.angle(V[i])).item()
        print(f"V{i+1} = {m:.4f} ∠ {a:.2f}°")
    if torch.max(torch.abs(mis)) < 1e-4:
        print("\nConverged.")
        break

    # Jacobian (4×4 for 2 PQ buses)
    J = torch.zeros(4, 4)
    for i in range(1, 3):
        for k in range(1, 3):
            Vi, Vk, Yik = V[i], V[k], Y[i, k]
            d = delta[i] - delta[k]
            if i == k:
                Pi, Qi = S_calc[i].real, S_calc[i].imag
                J[i-1, k-1] = -Qi - V_mag[i]**2 * Y[i,i].imag
                J[i-1, k+1] = Pi / V_mag[i] + V_mag[i] * Y[i,i].real
                J[i+1, k-1] = Pi - V_mag[i]**2 * Y[i,i].real
                J[i+1, k+1] = Qi / V_mag[i] - V_mag[i] * Y[i,i].imag
            else:
                J[i-1, k-1] = V_mag[i]*V_mag[k]*(Yik.real*torch.sin(d) - Yik.imag*torch.cos(d))
                J[i-1, k+1] = V_mag[i]*(Yik.real*torch.cos(d) + Yik.imag*torch.sin(d))
                J[i+1, k-1] = -V_mag[i]*V_mag[k]*(Yik.real*torch.cos(d) + Yik.imag*torch.sin(d))
                J[i+1, k+1] = V_mag[i]*(Yik.real*torch.sin(d) - Yik.imag*torch.cos(d))

    dx = torch.linalg.solve(J, mis)
    delta[1:] += dx[:2]
    V_mag[1:] += dx[2:]

# Final voltages
print("\nFinal Voltages (Polar Form):")
V = V_mag * torch.exp(1j * delta)
for i in range(3):
    m, a = V[i].abs().item(), torch.rad2deg(torch.angle(V[i])).item()
    print(f"V{i+1} = {m:.4f} ∠ {a:.2f}°")



Iter 1:
V1 = 1.0500 ∠ 0.00°
V2 = 1.0000 ∠ 0.00°
V3 = 1.0000 ∠ 0.00°

Iter 2:
V1 = 1.0500 ∠ 0.00°
V2 = 0.9842 ∠ -3.46°
V3 = 1.0032 ∠ -2.84°

Iter 3:
V1 = 1.0500 ∠ 0.00°
V2 = 0.9818 ∠ -3.50°
V3 = 1.0013 ∠ -2.86°

Iter 4:
V1 = 1.0500 ∠ 0.00°
V2 = 0.9818 ∠ -3.50°
V3 = 1.0012 ∠ -2.86°

Converged.

Final Voltages (Polar Form):
V1 = 1.0500 ∠ 0.00°
V2 = 0.9818 ∠ -3.50°
V3 = 1.0012 ∠ -2.86°
